# Bisection Method

In [1]:
%reset -f
import numpy as np
import typing

def bisection(
        func: typing.Callable[[float], float], 
        boundary: list, 
        maxiter: int = 500, 
        tol: float = 1e-5
        ) -> float:
    """
    Returns root of given function in given boundary found by bisection method.
    Function should be scalar-valued and single-variable.
    """
    a = boundary[0] # lower bound
    b = boundary[1] # upper bound
    
    assert func(a)*func(b) < 0, "No root in given range."

    if func(a)*func(b) == 0:
        if func(a) == 0:
            return a
        else:
            return b
    else:
        for _ in range(maxiter):
            m = (a + b)/2 # midpoint

            if abs(func(m)) < tol:
                return m
            
            if func(a)*func(m) > 0:
                a = m
            else: # func(b)*func(m) > 0
                b = m
        
        return m

In [2]:
# Example of bisection method
import numpy as np

func = lambda x: np.sin(x) - np.log(x)
root = bisection(func, [1, 10])
print(root)

2.219104766845703


# Newton-Raphson Method

In [1]:
%reset -f
import numpy as np
import typing

from differentiation import diff

def newton_raphson(
        func: typing.Callable[[float], float],
        diff_func: typing.Callable[[float], float] = None,
        x0: float = 0.1,
        tol: float = 1e-8,
        maxiter: int = 500,
    ) -> float:
    """
    Returns root of given function found by Newton-Raphson method.
    Function should be scalar-valued and single-variable.
    """
    if diff_func is None: # Numerically differentiate
        x = x0
        for _ in range(maxiter):
            if abs(func(x)) < tol:
                return x
            x -= func(x)/diff(func, x)

        return x
    else:
        x = x0
        for _ in range(maxiter):
            if abs(func(x)) < tol:
                return x
            x -= func(x)/diff_func(x)
        
        return x

In [4]:
# Example of Newton-Raphson method
import numpy as np

func = lambda x: np.sin(x) - np.log(x)
diff_func = lambda x: np.cos(x) - 1 / x
root = newton_raphson(func)
print(root)

2.2191071489140937


In [4]:
%reset -f
import numpy as np
import typing
from numpy import typing as nptyping

from differentiation import jacobi

def newton_raphson_multivar(
    func: typing.Callable[[nptyping.ArrayLike], nptyping.ArrayLike],
    x0: nptyping.ArrayLike,
    jacobian: typing.Callable[[nptyping.ArrayLike], np.ndarray] = None,
    tol: float = 1e-8,
    maxiter: int = 500,
) -> np.ndarray:
    if jacobian is None:
        x = x0
        for _ in range(maxiter):
            if np.linalg.norm(func(x)) < tol:
                return x
            jacobian = jacobi(func, x)            
            # Can use any system of equations solving method here
            delta_x = -np.matmul(np.linalg.inv(jacobian), func(x))
            
            x += delta_x
        print("Warning: Did not converge within the specified number of iterations.")
        return x
    else:
        x = x0

        for _ in range(maxiter):
            if np.linalg.norm(func(x)) < tol:
                return x
            # Can use any system of equations solving method here
            delta_x = -np.matmul(np.linalg.inv(jacobian(x)), func(x))
            
            x += delta_x
        return x


In [5]:
# Example of Newton-Raphson method for system of equations
def my_vector_function(X: nptyping.ArrayLike) -> np.ndarray:
    x = X[0]
    y = X[1]
    out1 = x * np.cos(y) - 4
    out2 = x * y - x - 5
    return np.array([out1, out2])


root = newton_raphson_multivar(my_vector_function, [1.0, 1.0])
print(root)

TypeError: 'float' object is not subscriptable

Code From ChatGPT

In [7]:
import numpy as np
from numpy.linalg import inv, norm


def numerical_jacobian(f, x, epsilon=1e-8):
    """
    Numerically calculate the Jacobian matrix of a vector function f with respect to x.

    Parameters:
        - f: The vector function, a callable that takes an n-dimensional vector x and returns an m-dimensional vector.
        - x: The input vector.
        - epsilon: The finite difference step size for numerical differentiation.

    Returns:
        - J: The Jacobian matrix of f with respect to x.
    """
    n = len(x)
    m = len(f(x))
    J = np.zeros((m, n))

    for i in range(n):
        x_plus_epsilon = x.copy()
        x_plus_epsilon[i] += epsilon
        J[:, i] = (f(x_plus_epsilon) - f(x)) / epsilon

    return J


def newton_raphson(f, x0, epsilon=1e-6, max_iter=100):
    """
    Perform the Newton-Raphson method for solving a system of nonlinear equations using numerical Jacobian.

    Parameters:
        - f: The vector function, a callable that takes an n-dimensional vector x and returns an n-dimensional vector.
        - x0: The initial guess for the solution.
        - epsilon: The finite difference step size for numerical Jacobian.
        - tol: Tolerance for convergence.
        - max_iter: Maximum number of iterations.

    Returns:
        - x: The solution vector.
        - num_iter: The number of iterations performed.
    """
    x = x0.copy()

    for i in range(max_iter):
        # Calculate the Jacobian matrix numerically
        J = numerical_jacobian(f, x, epsilon)

        print(J)

        # Solve the linear system J * delta_x = -f(x)
        delta_x = -np.dot(inv(J), f(x))

        # Update the solution
        x += delta_x

        # Check for convergence
        if norm(delta_x) < epsilon:
            return x, i + 1  # Return the solution and the number of iterations

    # If the method did not converge, raise a warning
    print(
        "Warning: Newton-Raphson method did not converge within the specified number of iterations."
    )

    return x, max_iter


# Example usage:
# Define your vector function f(x)
def my_vector_function(x):
    return np.array([x[0] ** 2 - 4, x[1] ** 2 - 1])


# Choose an initial guess
initial_guess = np.array([1.0, 1.0])

# Apply the Newton-Raphson method with numerical Jacobian
solution, iterations = newton_raphson(my_vector_function, initial_guess)

print("Solution:", solution)
print("Number of Iterations:", iterations)

[[2.000001 0.      ]
 [0.       2.000001]]
[[4.9999995 0.       ]
 [0.        2.000001 ]]
[[4.10000091 0.        ]
 [0.         2.000001  ]]
[[4.00122053 0.        ]
 [0.         2.000001  ]]
[[4.00000119 0.        ]
 [0.         2.000001  ]]
Solution: [2. 1.]
Number of Iterations: 5
